# BCG Finance AI Bot - Automated Data Extraction

This task involves Auto extraction of Financial Statements of Microsoft, Apple and Tesla for the fiscal year 2021, 2022 and 2023 respectively.

I wanted to perform data extraction by using two different menthods. 
Usually in companies the data is stored either in databased system or cloud system like AWS, GCP or Postreg.
Here, for this project I wanted to reteive the data from two different souces my first source would be extracting the data from local system in excel fomrat and scound I'm going to scrap the data from an online website using API.

My task for data extraction would be: 
1. Extracting the data from internet throught website
2. Mannually creating data files from the internet and store into local system
3. Performing data cleaning 


website_url = "https://www.sec.gov/edgar/search/#/dateRange=custom&category=form-cat1&startdt=2023-01-01&enddt=2026-02-24"

In [3]:
# Imporing All the Necessary Libraries
import requests
import pandas as pd

In [4]:
# Configuration for SEC API 
headers = {"User-Agent": "madhuraaravendekar29@gmail.com"}

fical_years = [2023,2024,2025]

cik_mapping = {
    "Tesla": "0001318605",
    "Apple": "0000789019",
    "Microsoft": "0000320193"
}

metrics = {
    "Total Revenue": "Revenues",
    "Net Income": "NetIncomeLoss",
    "Total Assets": "Assets",
    "Total Liabilities": "Liabilities",
    "Cash Flow": "NetCashProvidedByUsedInOperatingActivities"
}


## Fetching Data from SEC API for each company and each financial metric

def fetch_company_data(cik: str) -> dict:
    url = f"https://data.sec.gov/api/xbrl/companyfacts/CIK{cik}.json"
    response = requests.get(url, headers = headers)
    if response.status_code == 200:
        data = response.json()
        return data
    else:
        print(f"Failed to fetch data for CIK {cik}. Status code: {response.status_code}")



## Extracting Financial Metrics for each company and storing in a dictionary

def extract_financial_metrics(data: dict, metrics: dict, fical_years: list) -> dict:
    facts = data.get('facts', {}).get('us-gaap', {})
    print(facts)
    final_df = pd.DataFrame()

    for metric_name, tag in metrics.items():
        if tag not in facts:
            continue
        try:
            df = pd.DataFrame(facts[tag]['units']['USD'])
        except KeyError:
            continue
        df = df[df['form'] == '10-K']
        df['year'] = pd.to_datetime(df['end']).dt.year
        df = df[df['year'].isin(fical_years)]
        df = df.sort_values('end', ascending=False)
        df = df[['year', 'val']]
        df = df.rename(columns={'val': metric_name})
        df = df.drop_duplicates(subset='year', keep='first')

        if final_df.empty:
            final_df = df
        else:
            final_df = pd.merge(final_df, df, on='year', how='outer')

    return final_df


## Process Single Company Data and Store in a Dictionary

def process_company(company_name: str, cik: str) -> pd.DataFrame:
    print(f"Processing data for {company_name}........")
    raw_data = fetch_company_data(cik)
    company_df = extract_financial_metrics(raw_data, metrics, fical_years)
    company_df.insert(0, 'Company', company_name)
    return company_df


## Main Function to Process All Companies and Store in a Dictionary
def main():
    all_companies_df = pd.DataFrame()

    for company, cik in cik_mapping.items():
        company_df = process_company(company, cik)
        all_companies_df = pd.concat([all_companies_df, company_df], ignore_index=True)

    # Export to Excel
    output_file = "SEC_Financial_Data.xlsx"
    all_companies_df.to_excel(output_file, index=False)

    print("\n✅ Excel file downloaded successfully!")
    print(f"Saved as: {output_file}")
    print("\nPreview:")
    print(all_companies_df)


# --------------------------------------------------
# Run Script
# --------------------------------------------------

if __name__ == "__main__":
    main()

Processing data for Tesla........


IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)




✅ Excel file downloaded successfully!
Saved as: SEC_Financial_Data.xlsx

Preview:
     Company  year  Total Revenue    Net Income  Total Assets  \
0      Tesla  2023   9.677300e+10   14997000000  106618000000   
1      Tesla  2024   9.769000e+10    7091000000  122070000000   
2      Tesla  2025   9.482700e+10    3794000000  137806000000   
3      Apple  2023            NaN   72361000000  411976000000   
4      Apple  2024            NaN   88136000000  512163000000   
5      Apple  2025            NaN  101832000000  619003000000   
6  Microsoft  2023            NaN   96995000000  352583000000   
7  Microsoft  2024            NaN   93736000000  364980000000   
8  Microsoft  2025            NaN  112010000000  359241000000   

   Total Liabilities     Cash Flow  
0        43009000000   13256000000  
1        48390000000   14923000000  
2        54941000000   14747000000  
3       205753000000   87582000000  
4       243686000000  118548000000  
5       275524000000  136162000000  
6      

In [5]:
## Code to check a single company filing data from SEC API and extracting financial metrics for that company

headers = {"User-Agent": "Madhura Aravendekar madhuraaravendekar29@gmail.com"}

cik = {"Tesla": "0001318605", "Apple": "0000789019", "Microsoft": "0000320193"}
url = f"https://data.sec.gov/api/xbrl/companyfacts/CIK{cik['Microsoft']}.json"

response = requests.get(url, headers=headers)
data = response.json()

facts = data['facts']['us-gaap']
metrics = {
    "Total Revenue": "Revenues",
    "Net Income": "NetIncomeLoss",
    "Total Assets": "Assets",
    "Total Liabilities": "Liabilities",
    "Operating Cash Flow": "NetCashProvidedByUsedInOperatingActivities"
}

targeted_year = [2023,2024,2025]
final_df = pd.DataFrame()

for name, tag in metrics.items():
    if tag in facts:
        df = pd.DataFrame(facts[tag]['units']['USD'])
        df = df[df['form'] == '10-K']
        df['year'] = pd.to_datetime(df['end']).dt.year
        df = df[df['year'].isin(targeted_year)]
        df = df.sort_values('end', ascending=False)
        df = df[['year', 'val']]
        df = df.rename(columns={'val': name})
        df = df.drop_duplicates(subset='year', keep='first')
        if final_df.empty:
            final_df = df
        else:
            final_df = pd.merge(final_df, df, on='year', how='outer')




final_df

,year,Net Income,Total Assets,Total Liabilities,Operating Cash Flow
0,2023,96995000000,352583000000,290437000000,110543000000
1,2024,93736000000,364980000000,308030000000,118254000000
2,2025,112010000000,359241000000,285508000000,111482000000


In [6]:
# Get company specific all facts metrics name
headers = {"User-Agent": "Madhura Aravendekar madhuraaravendekar29@gmail.com"}
cik = '0000320193'
companyFacts = requests.get(
    f'https://data.sec.gov/api/xbrl/companyfacts/CIK{cik}.json',
    headers=headers
    )
companyFacts.json()['facts']['us-gaap'].keys()
# companyFacts.json()['facts']['us-gaap']['Revenues']['units']['USD'][-1]

dict_keys(['AccountsPayable', 'AccountsPayableCurrent', 'AccountsReceivableNetCurrent', 'AccruedIncomeTaxesCurrent', 'AccruedIncomeTaxesNoncurrent', 'AccruedLiabilities', 'AccruedLiabilitiesCurrent', 'AccruedMarketingCostsCurrent', 'AccumulatedDepreciationDepletionAndAmortizationPropertyPlantAndEquipment', 'AccumulatedOtherComprehensiveIncomeLossAvailableForSaleSecuritiesAdjustmentNetOfTax', 'AccumulatedOtherComprehensiveIncomeLossCumulativeChangesInNetGainLossFromCashFlowHedgesEffectNetOfTax', 'AccumulatedOtherComprehensiveIncomeLossForeignCurrencyTranslationAdjustmentNetOfTax', 'AccumulatedOtherComprehensiveIncomeLossNetOfTax', 'AdjustmentsToAdditionalPaidInCapitalSharebasedCompensationRequisiteServicePeriodRecognitionValue', 'AdjustmentsToAdditionalPaidInCapitalTaxEffectFromShareBasedCompensation', 'AdvertisingExpense', 'AllocatedShareBasedCompensationExpense', 'AllowanceForDoubtfulAccountsReceivableCurrent', 'AmortizationOfIntangibleAssets', 'AntidilutiveSecuritiesExcludedFromCompu

# BCG Finance AI Bot - Data Processing

### Importing Necessary Modules

In [10]:
import pandas as pd
import numpy as np 
import matplotlib as plt
import seaborn as sns

### Loading the automated fetch data from SEC EDGAR's database

In [12]:
## Loading the data fetch during the data extraction phase
df = pd.read_excel("SEC_Financial_Data.xlsx")
df

,Company,year,Total Revenue,Net Income,Total Assets,Total Liabilities,Cash Flow
0,Tesla,2023,9.677300e+10,14997000000,106618000000,43009000000,13256000000
1,Tesla,2024,9.769000e+10,7091000000,122070000000,48390000000,14923000000
2,Tesla,2025,9.482700e+10,3794000000,137806000000,54941000000,14747000000
3,Apple,2023,NaN,72361000000,411976000000,205753000000,87582000000
4,Apple,2024,NaN,88136000000,512163000000,243686000000,118548000000
5,Apple,2025,NaN,101832000000,619003000000,275524000000,136162000000
6,Microsoft,2023,NaN,96995000000,352583000000,290437000000,110543000000
7,Microsoft,2024,NaN,93736000000,364980000000,308030000000,118254000000
8,Microsoft,2025,NaN,112010000000,359241000000,285508000000,111482000000


###  Filling missing values for Total Revenue for Apple and Microsoft for the years 2023, 2024, and 2025 based on external data sources


In [14]:
df.isnull().sum()

Company              0
year                 0
Total Revenue        6
Net Income           0
Total Assets         0
Total Liabilities    0
Cash Flow            0
dtype: int64

In [15]:
manual_df = pd.DataFrame([
    {'Company': 'Apple', 'year': 2023, 'Total Revenue': 383285000000},
    {'Company': 'Apple', 'year': 2024, 'Total Revenue': 391035000000},
    {'Company': 'Apple', 'year': 2025, 'Total Revenue': 416161000000},
    {'Company': 'Microsoft', 'year': 2023, 'Total Revenue': 211915000000},
    {'Company': 'Microsoft', 'year': 2024, 'Total Revenue': 245122000000},
    {'Company': 'Microsoft', 'year': 2025, 'Total Revenue': 281724000000},
])

df = df.set_index(['Company', 'year'])
manual_df = manual_df.set_index(['Company', 'year'])

df.update(manual_df)

df = df.reset_index()
df

,Company,year,Total Revenue,Net Income,Total Assets,Total Liabilities,Cash Flow
0,Tesla,2023,9.677300e+10,14997000000,106618000000,43009000000,13256000000
1,Tesla,2024,9.769000e+10,7091000000,122070000000,48390000000,14923000000
2,Tesla,2025,9.482700e+10,3794000000,137806000000,54941000000,14747000000
3,Apple,2023,3.832850e+11,72361000000,411976000000,205753000000,87582000000
4,Apple,2024,3.910350e+11,88136000000,512163000000,243686000000,118548000000
5,Apple,2025,4.161610e+11,101832000000,619003000000,275524000000,136162000000
6,Microsoft,2023,2.119150e+11,96995000000,352583000000,290437000000,110543000000
7,Microsoft,2024,2.451220e+11,93736000000,364980000000,308030000000,118254000000
8,Microsoft,2025,2.817240e+11,112010000000,359241000000,285508000000,111482000000


### Checking the data types of the columns

In [17]:
df.dtypes

Company               object
year                   int64
Total Revenue        float64
Net Income             int64
Total Assets           int64
Total Liabilities      int64
Cash Flow              int64
dtype: object

In [18]:
df['Total Revenue'] = df['Total Revenue'].astype('int64')
df

,Company,year,Total Revenue,Net Income,Total Assets,Total Liabilities,Cash Flow
0,Tesla,2023,96773000000,14997000000,106618000000,43009000000,13256000000
1,Tesla,2024,97690000000,7091000000,122070000000,48390000000,14923000000
2,Tesla,2025,94827000000,3794000000,137806000000,54941000000,14747000000
3,Apple,2023,383285000000,72361000000,411976000000,205753000000,87582000000
4,Apple,2024,391035000000,88136000000,512163000000,243686000000,118548000000
5,Apple,2025,416161000000,101832000000,619003000000,275524000000,136162000000
6,Microsoft,2023,211915000000,96995000000,352583000000,290437000000,110543000000
7,Microsoft,2024,245122000000,93736000000,364980000000,308030000000,118254000000
8,Microsoft,2025,281724000000,112010000000,359241000000,285508000000,111482000000


### Calculating Year-by-Year growth rates for Total Revenue and Net Income 


In [20]:
df['Revenue Growth (%)'] = df.groupby(['Company'])['Total Revenue'].pct_change() * 100
df['Net Income Growth (%)'] = df.groupby(['Company'])['Net Income'].pct_change() * 100
df

,Company,year,Total Revenue,Net Income,Total Assets,Total Liabilities,Cash Flow,Revenue Growth (%),Net Income Growth (%)
0,Tesla,2023,96773000000,14997000000,106618000000,43009000000,13256000000,NaN,NaN
1,Tesla,2024,97690000000,7091000000,122070000000,48390000000,14923000000,0.947578,-52.717210
2,Tesla,2025,94827000000,3794000000,137806000000,54941000000,14747000000,-2.930699,-46.495558
3,Apple,2023,383285000000,72361000000,411976000000,205753000000,87582000000,NaN,NaN
4,Apple,2024,391035000000,88136000000,512163000000,243686000000,118548000000,2.021994,21.800417
5,Apple,2025,416161000000,101832000000,619003000000,275524000000,136162000000,6.425512,15.539621
6,Microsoft,2023,211915000000,96995000000,352583000000,290437000000,110543000000,NaN,NaN
7,Microsoft,2024,245122000000,93736000000,364980000000,308030000000,118254000000,15.669962,-3.359967
8,Microsoft,2025,281724000000,112010000000,359241000000,285508000000,111482000000,14.932156,19.495178


### Calculating Year-by-Year growth rates for Total Assets, Total Liabilities and Cash flow from Operations Activities

In [23]:
df['Assets Growth (%)'] = df.groupby('Company')['Total Assets'].pct_change() * 100
df['Liabilities Growth (%)'] = df.groupby('Company')['Total Liabilities'].pct_change() * 100
df['Cash Flow from Operations Growth(%)'] = df.groupby('Company')['Cash Flow'].pct_change() * 100

### Imputing Null/NaN values that result from pct_change calculations with 0

In [26]:
df.fillna(0, inplace=True)
df

,Company,year,Total Revenue,Net Income,Total Assets,Total Liabilities,Cash Flow,Revenue Growth (%),Net Income Growth (%),Assets Growth (%),Liabilities Growth (%),Cash Flow from Operations Growth(%)
0,Tesla,2023,96773000000,14997000000,106618000000,43009000000,13256000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,Tesla,2024,97690000000,7091000000,122070000000,48390000000,14923000000,0.947578,-52.717210,14.492862,12.511335,12.575438
2,Tesla,2025,94827000000,3794000000,137806000000,54941000000,14747000000,-2.930699,-46.495558,12.890964,13.537921,-1.179388
3,Apple,2023,383285000000,72361000000,411976000000,205753000000,87582000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,Apple,2024,391035000000,88136000000,512163000000,243686000000,118548000000,2.021994,21.800417,24.318650,18.436183,35.356580
5,Apple,2025,416161000000,101832000000,619003000000,275524000000,136162000000,6.425512,15.539621,20.860546,13.065174,14.858117
6,Microsoft,2023,211915000000,96995000000,352583000000,290437000000,110543000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,Microsoft,2024,245122000000,93736000000,364980000000,308030000000,118254000000,15.669962,-3.359967,3.516052,6.057424,6.975566
8,Microsoft,2025,281724000000,112010000000,359241000000,285508000000,111482000000,14.932156,19.495178,-1.572415,-7.311625,-5.726656


In [27]:
df.to_csv('SEC_data_cleaned.csv')